### IMPORTS

In [1]:
import subprocess
import os
import json

### CONSTANTS

In [2]:
BASE_DIR = "E:\Workspace\Work"

### CREATE FOLDER STRUCTURE

In [3]:
with open("./glab_groups.json") as f:
    groups = json.load(f)

for group in groups:
    # create group folder with full path
    group_folder = os.path.join(BASE_DIR, group["full_path"])
    os.makedirs(group_folder, exist_ok=True)

### CLONE GLAB PROJECTS INTO FOLDER STRUCTURE

In [5]:
with open("./glab_projects.json") as f:
    projects = json.load(f)


for project in projects:
    # create project folder with full path
    project_folder = os.path.join(BASE_DIR, project["path_with_namespace"])

    # check if the project is already cloned
    if os.path.exists(os.path.join(BASE_DIR, project["path_with_namespace"])):
        print(f"Project {project['name']} is already cloned, pulling latest changes...")
        # # run glab pull command
        # subprocess.run(
        #     f"cd {project_folder} && glab pull",
        #     shell=True,
        #     check=True,
        # )
    else:
        # run glab clone command
        subprocess.run(
            f"glab repo clone {project['path_with_namespace']} {project_folder}",
            shell=True,
            check=True,
        )
        print(f"Cloned project {project['name']} to {project_folder}")

Project RealEstateScrapeBot is already cloned, pulling latest changes...
Project OneTrueGit is already cloned, pulling latest changes...
Project MAT120 is already cloned, pulling latest changes...
Project CSE421 is already cloned, pulling latest changes...
Project CSE250 is already cloned, pulling latest changes...
Project CSE221 is already cloned, pulling latest changes...
Project CSE220 is already cloned, pulling latest changes...
Project CSE111 is already cloned, pulling latest changes...
Project CSE110 is already cloned, pulling latest changes...
Project The Library is already cloned, pulling latest changes...
Project jupyter-ipyreact is already cloned, pulling latest changes...
Project Data Structure is already cloned, pulling latest changes...
Project LAB-temp is already cloned, pulling latest changes...
Project BracU-App is already cloned, pulling latest changes...
Project BookStore is already cloned, pulling latest changes...
Project CarRepairDashboardApp is already cloned, pul

### GOTO EACH PROJECT AND CREATE A GITHUB REPO

#### ADD THE GITHUB REPO AS REMOTE
#### PUSH THE PROJECT TO GITHUB

In [20]:
allPathWithNamespace = [project["path_with_namespace"] for project in projects]


def convertToCamelCase(q: str):
    q.replace("/", "__")


print(*allPathWithNamespace, sep="\n")

breaktimecode/portfolio/webscraping/RealEstateScrapeBot
community3940225/onetruegit
bracu_23101281/lab/MAT120
bracu_23101281/lab/CSE421
bracu_23101281/lab/CSE250
bracu_23101281/lab/CSE221
bracu_23101281/lab/CSE220
bracu_23101281/lab/CSE111
bracu_23101281/lab/cse110
breaktimecode/reaserch-and-development/the-library
playground_23101281/jupyter-ipyreact
learning_23101281/data-structure-and-algorithms/data-structure
bracu_23101281/LAB-temp
breaktimecode/bracu-projects/bracu-app
bracu_23101281/group-projects/CSE111_SUM23_GroupProject
breaktimecode/portfolio/design/carrepairdashboardapp
